In [ ]:
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import figure
import numpy as np
from keras.models import load_model
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import glob
from keras.preprocessing import image as kimage
from roboflow import Roboflow
import json
from time import sleep
from PIL import Image, ImageDraw
import io
import base64
import random
import requests
from os.path import exists
import os, sys, re, glob
from IPython.display import clear_output
import os, urllib.request

# FFMPEG Set Up

### FFMPEG Installation

In [ ]:
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

Installation finished.


#### Access video

In [ ]:
%cd /content/
!mkdir videos_to_infer
!mkdir inferred_videos
!mkdir videos_to_infer_two
!mkdir inferred_videos_two

%cd videos_to_infer

/content
/content/videos_to_infer


#### Link your Google Drive to upload files to/from Google Drive


*   process outlined in the next 2 cells



In [ ]:
# OPTIONAL - link your g-drive to pull videos from
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# OPTIONAL - copy your videos from g-drive to /content/
!cp "/content/gdrive/MyDrive/Data 298B/main/training.mp4" "/content/videos_to_infer"

In [ ]:
!cp "/content/gdrive/MyDrive/Data 298B/main/test.mp4" "/content/videos_to_infer_two"

#### Break down video frames into images

In [ ]:
os.environ['inputFile'] = "/content/videos_to_infer/training.mp4"

!ffmpeg  -hide_banner -loglevel error -i "$inputFile" -vf fps=30 "$inputFile_out%04d.png"

In [ ]:
%cd 
%cd /content/videos_to_infer_two

/root
/content/videos_to_infer_two


In [ ]:
os.environ['inputFile_two'] = "/content/videos_to_infer_two/test.mp4"

!ffmpeg  -hide_banner -loglevel error -i "$inputFile_two" -vf fps=30 "$inputFile_out%04d.png"

# Main - Obj Detection

In [ ]:
# workspace code
from roboflow import Roboflow
import json

rf = Roboflow(api_key="7CZtbBaqqkv8yKNnDTR9")
project = rf.workspace().project("anomaly-detection-2.0")

# grab the model from that project's version
model = project.version(1).model
print(model)

loading Roboflow workspace...
loading Roboflow project...
{
  "id": "anomaly-detection-2.0/1",
  "name": "Anomaly detection 2.0",
  "version": "1",
  "classes": null,
  "overlap": 30,
  "confidence": 40,
  "stroke": 1,
  "labels": false,
  "format": "json",
  "base_url": "https://detect.roboflow.com/"
}


In [ ]:
from PIL import Image, ImageDraw, ImageFont

# HELPER FUNCTIONS BLOCK
def draw_boxes(box, x0, y0, img, class_name):
    # OPTIONAL - color map, change the key-values for each color to make the
    # class output labels specific to your dataset
    color_map = {
        "floor":"red",
        "door":"blue",
        "wall":"yellow",
        "person":"green"
    }

    # get position coordinates
    bbox = ImageDraw.Draw(img) 

    bbox.rectangle(box, outline =color_map[class_name], width=5)
    font = ImageFont.truetype("/content/gdrive/MyDrive/Data 298B/new/arial.ttf", 20) 
    bbox.text((x0, y0), class_name, fill='white', anchor=None , font = font, spacing = 20)

    return img

def save_with_bbox_renders(img):
    file_name = os.path.basename(img.filename)
    img.save('/content/inferred_videos/' + file_name)

In [ ]:
def add_text_to_image(image_path, text, position):
    # Open an Image
    img = image_path

    # Initialize ImageDraw
    draw = ImageDraw.Draw(img)

    # Specify Font 
    # This example uses the built-in "Arial" font
    # Make sure to have this font file (arial.ttf) in your working directory or give the absolute path
    font = ImageFont.truetype("/content/gdrive/MyDrive/Data 298B/new/arial.ttf", 20)

    # Add Text
    draw.text((10, position), text, fill="#25A1D4", font=font)

    # Save the Image
    return img

## Execution

In [ ]:
# set path and get all image files
file_path = "/content/videos_to_infer/"
extention = ".png"
globbed_files = sorted(glob.glob(file_path + '*' + extention))

# Prepare for training data
class_data = {"wall": [], "floor": [], "door": []}

# Iterate over all images
for img_path in globbed_files:
    predictions = model.predict(img_path, confidence=35, overlap=0).json()['predictions']
    newly_rendered_image = Image.open(img_path)

    for prediction in predictions:
        x0 = prediction['x'] - prediction['width'] / 2
        x1 = prediction['x'] + prediction['width'] / 2
        y0 = prediction['y'] - prediction['height'] / 2
        y1 = prediction['y'] + prediction['height'] / 2
        box = (x0, y0, x1, y1)

        # Crop each ROI and resize to 64x64 (or any size that your model accepts)
        roi = newly_rendered_image.crop(box).resize((64, 64))

        # Convert ROI to numpy array and normalize to [0,1]
        roi_np = np.array(roi) / 255.0
        class_data[prediction['class']].append(roi_np)

        # Store the ROI in the appropriate list depending on its class
        #if prediction['class'] == 'door':
            #train_data_door.append(roi_np)
        #elif prediction['class'] == 'wall':
            #train_data_wall.append(roi_np)
        #elif prediction['class'] == 'floor':
            #train_data_floor.append(roi_np)

        # Draw bounding boxes and add text to the image
        newly_rendered_image = draw_boxes(box, x0, y0, newly_rendered_image, prediction['class'])
        newly_rendered_image = add_text_to_image(newly_rendered_image, 'Status: Normal')

In [ ]:
def create_autoencoder():
    input_img = Input(shape=(64, 64, 3))
    l1_lambda = 0.001  # Define the lambda for L1 regularization

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(l1_lambda))(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(l1_lambda))(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(l1_lambda))(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(l1_lambda))(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same', kernel_regularizer=regularizers.l1(l1_lambda))(x)

    # Autoencoder = Encoder + Decoder
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# set path and get all image files
file_path = "/content/videos_to_infer_two/"
extention = ".png"
globbed_files = sorted(glob.glob(file_path + '*' + extention))
train_df = pd.read_csv('/content/gdrive/MyDrive/Data 298B/training_data.csv')

In [ ]:
# Convert list of numpy arrays to single numpy array
for class_name in class_data.keys():
    class_data[class_name] = np.array(class_data[class_name])

# Create and train an autoencoder for each class
autoencoders = {}
history_dict = {}

In [ ]:
# Preprocessing: convert categorical features if necessary
le = LabelEncoder()

train_df['location'] = le.fit_transform(df['location'])

# Splitting the dataset into the Training set and Test set
features = train_df.drop('target_label', axis=1)
target = train_df['target_label']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

# Creating and training the Random Forest model
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(features_train, target_train)

# Predicting the Test set results
target_pred = classifier.predict(features_test)

# Model accuracy
print('Model Accuracy is', accuracy_score(target_test, target_pred)*100)

# Saving the model
joblib.dump(classifier, 'random_forest_model.pkl')

In [ ]:
for class_name, data in class_data.items():
    print(f"Training autoencoder for {class_name}...")

    # Create an autoencoder for this class
    autoencoder = create_autoencoder()

    # Split the data into training and validation sets
    train_data = data[:int(len(data)*0.8)]
    val_data = data[int(len(data)*0.8):]

    # Train the autoencoder
    history = autoencoder.fit(train_data, train_data, epochs=500, batch_size=32, validation_data=(val_data, val_data), callbacks=[early_stopping_callback])

    # Store the trained autoencoder
    autoencoders[class_name] = autoencoder

    # Store the training history
    history_dict[class_name] = history

# Now we have a separate autoencoder for each class and we can use them to infer new images

# Save the models
for class_name, autoencoder in autoencoders.items():
    autoencoder.save(f"{class_name}_autoencoder.h5")

In [ ]:
def plot_history(history, title):
    loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']
    epochs = range(1, len(loss_values) + 1)

    plt.figure(figsize=(12, 6))
    plt.plot(epochs, loss_values, 'r', label='Training loss')
    plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation loss for ' + title + ' object')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
# Plot the history for 'door' class
plot_history(history_dict['door'], "door")
plot_history(history_dict['wall'], "wall")
plot_history(history_dict['floor'], "floor")